In [1]:
# --- Part 3: Practical Audit (COMPAS Recidivism Dataset) ---
# This script simulates the analysis of racial bias using the principles and metrics
# found in the IBM AI Fairness 360 (AIF360) toolkit.

import pandas as pd
import numpy as np
# In a real environment, you would import:
# from aif360.datasets import StandardDataset
# from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
# from aif360.algorithms.preprocessing import Reweighing

# --- 1. Simulate Loading and Pre-processing Data ---
# AIF360 requires specific column names for protected attributes and favorable labels.

# Create simulated COMPAS data based on ProPublica's findings
data_size = 1000
np.random.seed(42)

# Protected Attribute: Race (0=Black, 1=White)
race = np.random.choice([0, 1], size=data_size, p=[0.5, 0.5])

# Favorable Label: Non-Recidivism (0=Recidivate, 1=No Recidivate)
# Note: In the COMPAS context, a 'High Risk' score (Recidivate=0) is the unfavorable outcome.
true_labels = np.random.choice([0, 1], size=data_size, p=[0.45, 0.55])

# Simulated Prediction Score (0=High Risk/Recidivate, 1=Low Risk/No Recidivate)
# SIMULATED BIAS: Assume model is more likely to assign high risk (0) to Black individuals (0)
predictions = np.zeros(data_size)

# If race is Black (0): Assign Recidivism (0) 70% of the time, Low Risk (1) 30%
predictions[race == 0] = np.random.choice([0, 1], size=np.sum(race == 0), p=[0.70, 0.30])

# If race is White (1): Assign Recidivism (0) 40% of the time, Low Risk (1) 60%
predictions[race == 1] = np.random.choice([0, 1], size=np.sum(race == 1), p=[0.40, 0.60])

df = pd.DataFrame({
    'race': race,
    'true_label': true_labels,
    'prediction': predictions
})

# --- 2. Define Protected and Unprivileged Groups (AIF360 Structure) ---
# The 'favorable' outcome is Non-Recidivism (Low Risk), coded as 1.
# The 'unprivileged' group is Black (0).
privileged_groups = [{'race': 1}]  # White
unprivileged_groups = [{'race': 0}] # Black
favorable_label = 1

# --- 3. Calculate Bias Metrics (Simulated using Pandas) ---

# --- Disparate Impact Ratio (DIR) ---
# DIR = (Selection Rate for Unprivileged Group) / (Selection Rate for Privileged Group)
# Selection Rate = P(prediction=1)
rate_black = df[df['race'] == 0]['prediction'].mean()
rate_white = df[df['race'] == 1]['prediction'].mean()

dir_value = rate_black / rate_white if rate_white != 0 else 0

# --- Equal Opportunity Difference (EOD) ---
# EOD = (True Positive Rate for Unprivileged Group) - (True Positive Rate for Privileged Group)
# TPR (True Positive Rate) = P(prediction=1 | true_label=1) -- Correctly predicting Low Risk

# Calculate TPR for Black group (where true label is 1 / Low Risk)
black_low_risk_actual = df[(df['race'] == 0) & (df['true_label'] == favorable_label)]
tpr_black = black_low_risk_actual['prediction'].sum() / len(black_low_risk_actual) if len(black_low_risk_actual) > 0 else 0

# Calculate TPR for White group (where true label is 1 / Low Risk)
white_low_risk_actual = df[(df['race'] == 1) & (df['true_label'] == favorable_label)]
tpr_white = white_low_risk_actual['prediction'].sum() / len(white_low_risk_actual) if len(white_low_risk_actual) > 0 else 0

eod_value = tpr_black - tpr_white

# --- 4. Print Results ---
print("--- COMPAS Bias Audit Results (Racial Bias) ---")
print(f"Privileged Group (White Prediction Rate): {rate_white:.2f}")
print(f"Unprivileged Group (Black Prediction Rate): {rate_black:.2f}")
print(f"Disparate Impact Ratio (DIR): {dir_value:.2f}")

# Ideal DIR is 1.0 (Fairness range: 0.8 to 1.25).
if dir_value < 0.8 or dir_value > 1.25:
    print("STATUS: Bias Detected (Fails Statistical Parity)")
else:
    print("STATUS: Fair (Passes Statistical Parity)")

print("\n--- Equal Opportunity (Recidivism) ---")
print(f"True Positive Rate (TPR) for Black Group: {tpr_black:.2f}")
print(f"True Positive Rate (TPR) for White Group: {tpr_white:.2f}")
print(f"Equal Opportunity Difference (EOD): {eod_value:.2f}")

# Ideal EOD is 0.0.
if abs(eod_value) > 0.1: # Threshold for fairness
    print("STATUS: Significant Bias Detected (Fails Equal Opportunity)")
else:
    print("STATUS: Acceptable (Passes Equal Opportunity)")

# --- 5. Remediation Concept (If this were a real AIF360 application) ---
# remediation = Reweighing(unprivileged_groups=unprivileged_groups,
#                          privileged_groups=privileged_groups)
# remediated_data = remediation.fit_transform(data)
# model.fit(remediated_data)
print("\nRemediation Step: Reweighing (Pre-processing mitigation) would be applied to assign")
print("higher weights to the underrepresented or unfairly treated subgroups in the training data.")

--- COMPAS Bias Audit Results (Racial Bias) ---
Privileged Group (White Prediction Rate): 0.59
Unprivileged Group (Black Prediction Rate): 0.29
Disparate Impact Ratio (DIR): 0.49
STATUS: Bias Detected (Fails Statistical Parity)

--- Equal Opportunity (Recidivism) ---
True Positive Rate (TPR) for Black Group: 0.32
True Positive Rate (TPR) for White Group: 0.56
Equal Opportunity Difference (EOD): -0.24
STATUS: Significant Bias Detected (Fails Equal Opportunity)

Remediation Step: Reweighing (Pre-processing mitigation) would be applied to assign
higher weights to the underrepresented or unfairly treated subgroups in the training data.
